In [ ]:
import sys
sys.path.append('/home/sensei/jupy-notebooks/Analytics/PorterFarms/')
print("============================================")
print("/  MoteHealth is running.                  /")
print("============================================")

import requests
from datetime import datetime, timedelta
import pytz
from slackclient import SlackClient
import json
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser

config = configparser.ConfigParser()
config.read("../../../analytics_secrets.ini")

_ACTIVE_STANDBY = config['DEFAULT']['role']
if _ACTIVE_STANDBY == 'STANDBY':
    print("STANDBY")
    raise SystemExit("Stop right there!")
else:
  _SLACK_TOKEN = config['slack']['token']
  _CHIRPSTACK_HOST = config['chirpstack']['host']
  _CHIRPSTACK_USER = config['chirpstack']['user']
  _CHIRPSTACK_PASS = config['chirpstack']['password']
  _DB_HOST  = config['kanjidb']['dbhost']
  _DB_PORT  = config['kanjidb']['dbport']
  _DB_NAME  = config['kanjidb']['dbname']
  _DB_USER  = config['kanjidb']['dbuser']
  _DB_PASS  = config['kanjidb']['dbpass']

  _SLACK_SYSTEMCHANNEL_NAME = config['analytics']['systemchannelname']
  _SLACK_SYSTEMCHANNEL_DBID = int(config['analytics']['systemchannelid'])

  _USE_DROPBOX   = config['dropbox']['usedropbox']
  _FALLBACK_IMAGE = config['dropbox']['fallbackimage']

  _UTC_OFFSET = int(config['DEFAULT']['utcoffset'])

  _LOG_DEBUG = 0
  _LOG_INFO  = 1
  _LOG_ERROR = 2
  _LOG_LEVEL = int(config['DEFAULT']['loglevel'])
def logger(level, message):
    if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

import kanjiticketing as kt

conn = kt.getKanjiDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  print("Welcome to Jupyter Notebook.  You are connected to the Kanji database!")
else:
  print("You are not connected to the database.")

messagetemplate = "[\
   {\"type\": \"section\", \
		\"text\": { \
			\"type\": \"mrkdwn\", \
			\"text\": \"*<fakeLink.toUserProfiles.com|Iris / Zelda 1-1>*\\nTuesday, January 21 4:00-4:30pm\\nBuilding 2 - Havarti Cheese (3)\\n2 guests\" \
		}, \
		\"accessory\": { \
			\"type\": \"image\", \
			\"image_url\": \"https://api.slack.com/img/blocks/bkb_template_images/notifications.png\", \
			\"alt_text\": \"calendar thumbnail\" \
		} \
   } ]"

In [8]:
def postChirpStack(url, payload):
  try:
    response = requests.post(url, json=payload)
    if response.status_code == 200:      
      response = {"code":200, "payload": response.json()}
      return response
    
    else:
      blockmessage = json.loads(messagetemplate)
      # fixup message content
      blockmessage[0]["accessory"]["image_url"] = "https://www.dropbox.com/s/vlvnokujfx67zav/porter-farms.jpg?raw=1"
      blockmessage[0]["text"]["text"] = "*ChirpStack API non-200 POST response on {}.* infr_PANIC".format(url)
      #
      postMessageToSlack(blockmessage)  
  except requests.exceptions.RequestException as e: 
    blockmessage = json.loads(messagetemplate)
    # fixup message content
    blockmessage[0]["accessory"]["image_url"] = "https://www.dropbox.com/s/vlvnokujfx67zav/porter-farms.jpg?raw=1"
    blockmessage[0]["text"]["text"] = "*ChirpStack API POST request exception on {}.* infr_PANIC".format(url)
    #
    postMessageToSlack(blockmessage)  


In [9]:
def getChirpStack(url, headers):
    try:
      response = requests.get(url = url, headers = headers)
      if response.status_code == 200:
        response = {"code":200, "payload": response.json()}        
        return response    
      else:
        response = {"code":501}
        return response
    except requests.exceptions.RequestException as e: 
      response = {"code":502}
      return response

In [10]:
logger(_LOG_DEBUG, "{} {}".format(_CHIRPSTACK_USER, _CHIRPSTACK_PASS))

payload = {"password": _CHIRPSTACK_PASS, "username": _CHIRPSTACK_USER}
logger(_LOG_DEBUG, payload)

loginUrl = 'http://{}:8080/api/internal/login'.format(_CHIRPSTACK_HOST)

response = postChirpStack(loginUrl, payload)

if response["code"] == 200:
    chirpStackToken = response['payload']['jwt']
    logger(_LOG_DEBUG, chirpStackToken)   


In [13]:
# Reissue Alerts on OPEN tickets every 15minutes
_TICKETAGE_REISSUE_THRESHOLD_SECONDS = 15 * 60

_PREFERRED_IMAGE =  "https://www.dropbox.com/s/flyz08qx9h4ow52/missing-mote.jpg?raw=1"

if _USE_DROPBOX == 'true':
  locationimageurl = _PREFERRED_IMAGE
else:
  locationimageurl = _FALLBACK_IMAGE

# If mote is not seen for 30minutes, generate a ticket
_MAX_MOTE_AGE_SECONDS = 30 *60

#Ticket Type
MOTE_MISSING = 10003

#Mote types
_MAINS_MOTE_TYPE = 10000
_BATTERY_MOTE_TYPE = 10002

ticketnow = datetime.now(pytz.utc)  #tz Aware
now = datetime.now() - timedelta(hours = _UTC_OFFSET)

motequery = "SELECT * FROM kanji_node WHERE deploystate_id=10001 AND (nodetype_id={} OR nodetype_id={})".format(_MAINS_MOTE_TYPE, _BATTERY_MOTE_TYPE)
logger(_LOG_DEBUG, motequery)
df = pd.read_sql(motequery, conn)

logger(_LOG_DEBUG, "number of motes {}".format(len(df.index)))
headers = {"Grpc-Metadata-Authorization": "Bearer {}".format(chirpStackToken)} 
for ind in df.index:
    moteid = df['deveui'][ind]
    logger(_LOG_DEBUG, moteid )
    # chirpstack gateway api-endpoint 
    url = "http://{}:8080/api/devices/{}".format(_CHIRPSTACK_HOST, moteid)
    logger(_LOG_DEBUG, url)
    response = getChirpStack(url, headers)
    if response["code"] == 200:
      # extracting data in json format 
      mote = response["payload"]
      motedetails = mote["device"]
      age = now - datetime.strptime(mote['lastSeenAt'], '%Y-%m-%dT%H:%M:%S.%fZ')
      ageseconds =  age.days*24*60*60 + age.seconds
      moteinfo = {"name" : motedetails['name'],
                     "lastseen" : mote['lastSeenAt'],
                     "ageseconds" : ageseconds}     
      keyword = "infr_INFO"
      if ageseconds > _MAX_MOTE_AGE_SECONDS:
        node_id = df['idnode'][ind]
        nodename = df['name'][ind]
        location_id = df['location_id'][ind]
        locationquery = "SELECT location.idlocation, location.description, location.imageurl, location.slackalertchannel_id, \
                         customer.slacktoken, slackchannel.idslackchannel, slackchannel.channelname, slackchannel.channelid\
                         FROM kanji_location location \
                         JOIN kanji_customer customer ON location.customer_id=customer.idcustomer \
                         JOIN kanji_slackchannel slackchannel ON customer.slacksystemticketchannel_id=slackchannel.idslackchannel \
                         WHERE idlocation={}".format(location_id)
        
        df3 = pd.read_sql(locationquery, conn)
        locationid = df3["idlocation"][0]  
       
        logger(_LOG_DEBUG, "ImageUrl: {}".format(locationimageurl))        
        locationdescription = df3["description"][0]
        logger(_LOG_DEBUG, "Location: {}".format(locationdescription))
        
        #_SLACK_TOKEN = df3["slacktoken"][0] 
        #_SLACK_CHANNEL_NAME = df3["channelname"][0]
        #_SLACK_CHANNEL_ID = df3["channelid"][0]
        #_SLACK_CHANNEL_DBID = df3["idslackchannel"][0]
        
        description = "LoRa mote is missing. (last heard {}mins ago)".format(int(ageseconds/60))
        mentions = ""
        #generate and Slack a new ticket ONLY if there is not a currently open ticket for this issue
        openTicket = kt.ticketExists(conn, node_id, MOTE_MISSING, [kt._OPEN_STATUS, kt._WORKING_STATUS])
        if openTicket is None:
          ticketid = kt.openticket(conn, node_id, locationid, description, 2, 3, MOTE_MISSING, _SLACK_SYSTEMCHANNEL_DBID)
          ts = kt.slackticket(nodename, locationdescription, description, mentions, 2, 3, locationimageurl, _SLACK_TOKEN, _SLACK_SYSTEMCHANNEL_NAME, ticketid, 0)
          kt.updateTicket(conn, ticketid, ts)  
          logger(_LOG_INFO, "New ticket {} created for this issue.".format(ticketid))
        else:
          logger(_LOG_DEBUG, "There is an existing ticket {} for this issue. {}".format(openTicket['idticket'][0], openTicket['opentimestamp'][0]))
      else:
        status = "Ok"
        #is there an OPEN ticket on this mote?
      logger(_LOG_INFO, moteinfo)
logger(_LOG_INFO, "MoteHealth Done!")  

{'name': 'agMote-20002', 'lastseen': '2020-06-15T16:28:42.421539Z', 'ageseconds': 228}
{'name': 'agMote-20009', 'lastseen': '2020-06-15T14:40:07.831764Z', 'ageseconds': 6743}
{'name': 'agMote-20010', 'lastseen': '2020-06-15T16:32:01.741644Z', 'ageseconds': 29}
{'name': 'agMote-20000', 'lastseen': '2020-06-15T16:32:08.120070Z', 'ageseconds': 22}
{'name': 'agMote-20003', 'lastseen': '2020-06-15T14:42:29.534825Z', 'ageseconds': 6601}
MoteHealth Done!
